In [ ]:
!ls "/content/drive/MyDrive/AI Project: Bear Classifire/bear-dataset"

In [ ]:
data_path = "/content/drive/MyDrive/AI Project: Bear Classifire/bear-dataset"

img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load MobileNetV2 without the top classification layer
base = MobileNetV2(weights='imagenet', include_top=False)

# Add your custom layers
x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
out = Dense(5, activation='softmax')(x)  # 5 classes (5 bear types)

model = Model(base.input, out)

# Freeze the pretrained layers
for layer in base.layers:
    layer.trainable = False

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

In [ ]:
model.save("/content/bear_classifier.h5")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

labels = train_data.class_indices
# Convert: { 'black_bear':0 } → [ 'black_bear', ...]
labels = list(labels.keys())

def classify(img_input):
    # Ensure img_input is a PIL Image (Gradio usually passes it as such)
    # Convert to NumPy array and then back to PIL Image to break references
    # This creates a completely new image object with its own data.
    img_np = np.array(img_input)
    img_pil = Image.fromarray(img_np)

    # Resize the PIL Image using the predefined img_size
    img_resized = img_pil.resize(img_size)

    # Convert to array and normalize
    img_array = image.img_to_array(img_resized) / 255.0
    img_expanded = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_expanded)[0]
    return {labels[i]: float(prediction[i]) for i in range(len(labels))}

gr.Interface(fn=classify, inputs="image", outputs="label").launch(debug=True)